In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
# from census import Census
from us import states
import json

#pip istall only needs to be ran once then this code can be removed
!pip install -U epiweeks
import epiweeks as ew

# Census API Key
from config import api_key
# c = Census(api_key)
c = api_key

Requirement already up-to-date: epiweeks in c:\users\superpawn\anaconda3\lib\site-packages (2.1.2)


In [2]:
#Disease CSV data: issues are traslating 'epiweek' column into a year, data frame will need to be shortened to only inlude years 2005-2010

disease_df = pd.read_csv("diseases_us_1916_to_2011.csv")

disease_df['disease'].value_counts()

C:\Users\Superpawn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


DIPHTHERIA     158983
MEASLES        145167
PERTUSSIS      109072
HEPATITIS A     90839
POLIO           81531
MUMPS           69754
RUBELLA         53205
SMALLPOX        50916
Name: disease, dtype: int64

In [21]:
disease_df["cases"]=disease_df["cases"].astype(str).replace("\\N",np.nan)
disease_df=disease_df.dropna(how="any")
disease_df["cases"].astype(int)
# disease_df.dtypes

0         3
1         1
2         6
3         2
4         1
         ..
759462    0
759463    0
759464    1
759465    2
759466    0
Name: cases, Length: 759420, dtype: int32

In [4]:
#Convert epi_week column into week and years columns.
disease_df['epi_week'] = disease_df['epi_week'].astype(str)
disease_df['year'] = disease_df['epi_week'].str[0:4]
disease_df['week'] = disease_df['epi_week'].str[4:6]
disease_df

,epi_week,state,loc,loc_type,disease,cases,incidence_per_100000,year,week
0,196601,MN,MINNESOTA,STATE,HEPATITIS A,3,0.08,1966,01
1,196601,CO,COLORADO,STATE,HEPATITIS A,1,0.05,1966,01
2,196601,AZ,ARIZONA,STATE,HEPATITIS A,6,0.37,1966,01
3,196601,MT,MONTANA,STATE,HEPATITIS A,2,0.28,1966,01
4,196601,LA,LOUISIANA,STATE,HEPATITIS A,1,0.03,1966,01
...,...,...,...,...,...,...,...,...,...
759462,194752,WI,SUPERIOR,CITY,DIPHTHERIA,0,0.00,1947,52
759463,194752,FL,TAMPA,CITY,DIPHTHERIA,0,0.00,1947,52
759464,194752,CO,DENVER,CITY,DIPHTHERIA,1,0.25,1947,52
759465,194752,NY,BUFFALO,CITY,DIPHTHERIA,2,0.35,1947,52


In [20]:
short_disease_df = disease_df.loc[(disease_df['year']== '2010')|(disease_df['year']== '2009')|(disease_df['year']== '2008')|(disease_df['year']== '2007')|(disease_df['year']== '2006')]

grouped_disease_df = short_disease_df.groupby(['year','state','disease',])
grouped_disease_df['cases'].sum()

year  state  disease    
2006  AK     PERTUSSIS       112721213011020101200201030202151122001000001001
      AL     HEPATITIS A           110000000000001300011000022001000010102000
             PERTUSSIS      1120202100121101100100310143110643000160452521...
      AR     HEPATITIS A            35011001020100100010011001002010000100000
             PERTUSSIS      2101140302210510210142001111021003012160131200000
                                                  ...                        
2010  WI     PERTUSSIS      0004000000000000002018160251415091132619221960...
      WV     HEPATITIS A           000000000000000000000001000000000050010002
             PERTUSSIS      1000000012011010000001010021443116183942631001...
      WY     HEPATITIS A                        30000000000000000000000000000
             PERTUSSIS            0000000000010010010001000010100000000000000
Name: cases, Length: 503, dtype: object

In [7]:
#Below is the alternate way to pull census income data from the ACS5 using the URL

In [8]:
year=2010
target_url = f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,B01003_001E,B19013_001E&for=state:*&key={api_key}"
income_data = requests.get(target_url).json()

In [9]:
#print the Json data with formatting
print(json.dumps(income_data, indent=4, sort_keys=True))

[
    [
        "NAME",
        "B01003_001E",
        "B19013_001E",
        "state"
    ],
    [
        "Alabama",
        "4712651",
        "42081",
        "01"
    ],
    [
        "Alaska",
        "691189",
        "66521",
        "02"
    ],
    [
        "Arizona",
        "6246816",
        "50448",
        "04"
    ],
    [
        "Arkansas",
        "2872684",
        "39267",
        "05"
    ],
    [
        "California",
        "36637290",
        "60883",
        "06"
    ],
    [
        "Colorado",
        "4887061",
        "56456",
        "08"
    ],
    [
        "Connecticut",
        "3545837",
        "67740",
        "09"
    ],
    [
        "Delaware",
        "881278",
        "57599",
        "10"
    ],
    [
        "District of Columbia",
        "584400",
        "58526",
        "11"
    ],
    [
        "Florida",
        "18511620",
        "47661",
        "12"
    ],
    [
        "Georgia",
        "9468815",
        "49347",
        "13"
  

In [10]:
#reading the census Json data into a dataframe. issues include translating the state names into state abreviations to match the disease data frame, dropping the frst row, and renaming the columns
income_df = pd.DataFrame(income_data)
income_df 

,0,1,2,3
0,NAME,B01003_001E,B19013_001E,state
1,Alabama,4712651,42081,01
2,Alaska,691189,66521,02
3,Arizona,6246816,50448,04
4,Arkansas,2872684,39267,05
5,California,36637290,60883,06
6,Colorado,4887061,56456,08
7,Connecticut,3545837,67740,09
8,Delaware,881278,57599,10
9,District of Columbia,584400,58526,11


In [11]:
#Firt row dropped
income_df.drop(index=income_df.index[0],axis=0,inplace=True)
income_df

,0,1,2,3
1,Alabama,4712651,42081,01
2,Alaska,691189,66521,02
3,Arizona,6246816,50448,04
4,Arkansas,2872684,39267,05
5,California,36637290,60883,06
6,Colorado,4887061,56456,08
7,Connecticut,3545837,67740,09
8,Delaware,881278,57599,10
9,District of Columbia,584400,58526,11
10,Florida,18511620,47661,12
